In [1]:
!pip install ollama

In [93]:
from pathlib import Path

roles = ["sys","human"]
levels = ["hard", "medium", "easy"]
question_format_template   = Path("question_format_template.txt").read_text().strip()

prompt = {}
for role in roles:
    prompt[role] = {}
    for level in levels:
        prompt_text = Path(role + "_template_"+level+".txt").read_text().strip()
        if (role == "human"):
            prompt_text += question_format_template
        prompt[role][level] = prompt_text

In [94]:
import json
import requests

def chat(server,role, message):
    messages = [
    {
        'role': role,
        'content': message,
    },
    ]
    r = requests.post(
        server + "/api/chat",
        json={"model": "llama3",  "messages": messages, "stream": True},
    )
    r.raise_for_status()
    output = ""

    for line in r.iter_lines():
        body = json.loads(line)
        if "error" in body:
            raise Exception(body["error"])
        if body.get("done") is False:
            response = body.get("message", "")
            content = response.get("content", "")
            output += content
            # the response streams one token at a time, print that as we receive it
            #print(content, end="", flush=True)

        if body.get("done", False):
            response["content"] = output
            return output

In [95]:
import json
def parse_question(chat_response):
    response = chat_response.replace("\n","")
    json_string = response.split("{")[1].split("}")[0]
    question = json.loads("{" + json_string + "}")
    return question

In [96]:
def create_question(server,context, level):
    chat(server,"system",prompt["sys"][level])
    response = chat(server,"user",prompt["human"][level].replace('{context}',context))
    return parse_question(response)

In [97]:
import json
sample = context = "La Wikipedia en español es la edición en español o castellano de Wikipedia. Al igual que las versiones existentes de Wikipedia en otros idiomas, es una enciclopedia de contenido libre, publicada en Internet bajo las licencias libres CC BY-SA 4.0 y GFDL. En la actualidad cuenta con 1 949 592 artículos, y es escrita por usuarios voluntarios, es decir, que cualquiera puede editar un artículo, corregirlo o ampliarlo. Los servidores son administrados por la Fundación Wikimedia, una organización sin ánimo de lucro cuya financiación se basa fundamentalmente en donaciones."
levels = ["hard", "medium", "easy"] 


server = "http://0.0.0.0:11434" #local
#server = "https://librairy.linkeddata.es/ollama" #remote

for level in levels:
    print("making a " + level + " question ..")
    question = create_question(server,sample, level)
    print(json.dumps(question,indent=2, ensure_ascii=False))


making a hard question ..
{
  "PREGUNTA": "¿Cuál es el nombre de la licencia bajo la que se publican los contenidos de Wikipedia en español?",
  "OPCION1": "CC BY-NC-SA",
  "OPCION2": "GFDL-SAS",
  "OPCION3": "GPL v3.0",
  "OPCION4": "CC BY-SA 4.0 y GFDL",
  "EVIDENCIA": "La licencia CC BY-SA 4.0 y GFDL se especifica en el texto como las que rigen la publicación de los contenidos de Wikipedia en español."
}
making a medium question ..
{
  "PREGUNTA": "¿Quién financia el trabajo en Wikipedia?",
  "OPCION1": "La Fundación Wikimedia",
  "OPCION2": "Los usuarios que editan",
  "OPCION3": "El gobierno español",
  "OPCION4": "Fundación Wikimedia y donaciones de usuarios",
  "EVIDENCIA": "Según el texto, la financiación de la Fundación Wikimedia se basa fundamentalmente en donaciones."
}
making a easy question ..
{
  "PREGUNTA": "Cuántos artículos tiene la Wikipedia en español?",
  "OPCION1": "1000",
  "OPCION2": "500",
  "OPCION3": "2000",
  "OPCION4": "1949 592",
  "EVIDENCIA": "La respuest